In [27]:
import numpy as np

from manipulation import running_as_notebook

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    DiagramBuilder,
    ModelVisualizer,
    MultibodyPlant,
    BodyIndex,
    InverseKinematics,
    RotationMatrix,
    SceneGraph,
    Simulator,
    RollPitchYaw,
    InverseDynamicsController,
    BsplineTrajectory,
    KinematicTrajectoryOptimization,
    SpatialVelocityConstraint,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MinimumDistanceLowerBoundConstraint,
    Parser,
    PositionConstraint,
    Rgba,
    RigidTransform,
    Role,
    Solve,
    Sphere,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad, PublishPositionTrajectory
from manipulation.utils import ConfigureParser
from centers import centers
from MIDI.midi_to_hit import midi_to_hit

from IPython.display import clear_output

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [28]:
# robot_directives = """
# directives:

# - add_model:
#     name: iiwa_right
#     file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
#     default_joint_positions:
#         iiwa_joint_1: [-1]
#         iiwa_joint_2: [0.1]
#         iiwa_joint_3: [0]
#         iiwa_joint_4: [-1.2]
#         iiwa_joint_5: [0]
#         iiwa_joint_6: [ 1.6]
#         iiwa_joint_7: [10]
# - add_weld:
#     parent: world
#     child: iiwa_right::iiwa_link_0
#     X_PC:
#         translation: [0.4, 0.6, 0]
#         rotation: !Rpy { deg: [0, 0, -0]}
# - add_model:
#     name: drum_stick_right
#     file: package://robotics_final_project/drum_sticks.sdf
# - add_weld:
#     parent: iiwa_right::iiwa_link_7
#     child: drum_stick_right::stick
#     X_PC:
#         translation: [0, 0, 0.18]
#         rotation: !Rpy { deg: [180, 0, 0]}
# - add_model:
#     name: drum_kit
#     file: package://robotics_final_project/drum_kit.sdf

# """
robot_directives = """
directives:

- add_model:
    name: iiwa_right
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [10]
- add_weld:
    parent: world
    child: iiwa_right::iiwa_link_0
    X_PC:
        translation: [0.4, 0.6, 0]
        rotation: !Rpy { deg: [0, 0, -0]}
- add_model:
    name: drum_stick_right
    file: package://robotics_final_project/drum_sticks.sdf
- add_weld:
    parent: iiwa_right::iiwa_link_7
    child: drum_stick_right::stick
    X_PC:
        translation: [0, 0, 0.18]
        rotation: !Rpy { deg: [180, 0, 0]}

- add_model:
    name: iiwa_left
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [10]
- add_weld:
    parent: world
    child: iiwa_left::iiwa_link_0
    X_PC:
        translation: [0.7, -0.6, 0]
        rotation: !Rpy { deg: [0, 0, -0]}
- add_model:
    name: drum_stick_left
    file: package://robotics_final_project/drum_sticks.sdf
- add_weld:
    parent: iiwa_left::iiwa_link_7
    child: drum_stick_left::stick
    X_PC:
        translation: [0, 0, 0.18]
        rotation: !Rpy { deg: [180, 0, 0]}

- add_model:
    name: drum_kit
    file: package://robotics_final_project/drum_kit.sdf

"""

In [29]:
stiffness=20000
dissipation=2

meshcat.Delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=0.00001
    )

parser = Parser(plant)
ConfigureParser(parser)

parser.package_map().Add("robotics_final_project", "./")
parser.AddModelsFromString(robot_directives, ".dmd.yaml")
plant.Finalize()

visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(role=Role.kIllustration),
)

collision_visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(
        prefix="collision", role=Role.kProximity, visible_by_default=False
    ),
)

kp = [100] * plant.num_positions()
ki = [1] * plant.num_positions()
kd = [20] * plant.num_positions()
iiwa_controller = builder.AddSystem(
    InverseDynamicsController(plant, kp, ki, kd, False)
)
iiwa_controller.set_name("iiwa_controller")

builder.Connect(
    plant.get_state_output_port(),
    iiwa_controller.get_input_port_estimated_state(),
)
builder.Connect(
    iiwa_controller.get_output_port_control(), plant.get_actuation_input_port()
)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)

# meshcat.StartRecording()
# simulator.AdvanceTo(5.0)

# meshcat.StopRecording() 
# meshcat.PublishRecording()

In [30]:
import numpy as np
from MIDI.midi_to_name import midi_to_name, hit_mapping
from mido import MidiFile, MetaMessage, tick2second
hits = midi_to_hit(filename = 'MIDI/midi_drums.mid') #midi_drums
print(hits)


[{'hit': 'hi_hat', 'last_hit_time': 0.0010416666666666667, 'total_time': 0.0010416666666666667, 'strength': 114}, {'hit': 'hi_hat', 'last_hit_time': 0.25625, 'total_time': 0.25729166666666664, 'strength': 50}, {'hit': 'snare', 'last_hit_time': 0.23958333333333334, 'total_time': 0.496875, 'strength': 97}, {'hit': 'hi_hat', 'last_hit_time': 0.009375, 'total_time': 0.50625, 'strength': 117}, {'hit': 'hi_hat', 'last_hit_time': 0.2510416666666666, 'total_time': 0.7572916666666667, 'strength': 41}, {'hit': 'hi_hat', 'last_hit_time': 0.13333333333333333, 'total_time': 0.890625, 'strength': 80}, {'hit': 'hi_hat', 'last_hit_time': 0.125, 'total_time': 1.015625, 'strength': 107}, {'hit': 'hi_hat', 'last_hit_time': 0.253125, 'total_time': 1.26875, 'strength': 62}, {'hit': 'snare', 'last_hit_time': 0.22291666666666665, 'total_time': 1.491666666666667, 'strength': 97}, {'hit': 'hi_hat', 'last_hit_time': 0.02395833333333333, 'total_time': 1.5156250000000002, 'strength': 117}, {'hit': 'hi_hat', 'last

In [31]:
import copy

left = ["hi_hat", "snare", "crash", "hi_tom"]
right = ["mid_tom", "ride","mid_tom2", "ride2","low_tom2","low_tom"]
# left = ["hi_hat", "hi_hat2"]
# right = ["snare","snare2"]
def left_right_detector(hit):
    if hit["hit"] in left:
        side = "left"
    else:
        side = "right"
    return side

newhits=[]
def timecalc(hit1, hit2):
    halftime = 0.5 * (hit2 +hit1)
    return halftime

for i in range(len(hits)):
    if left_right_detector(hits[i]) == "right":
        if i ==0:
            halftime = timecalc(0, hits[i]["total_time"])
            print(halftime)
        else:
            halftime = timecalc(hits[i-1]["total_time"], hits[i]["total_time"])
        l = hits[i]
        ll= copy.copy(l)
        newhits.append(ll)
        # print()
        ll["hit"] = ll["hit"] + "2"
        ll["total_time"] = halftime 
        newhits.append(l)
    # print(newhits[-1]["total_time"])
newhits

[]

In [32]:

def visualize_frame(name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies

        Input:
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.

        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )

In [33]:
meshcat.StartRecording()


avoid_collisions  = True 

for hit in hits[:-8]:
    print(hit["hit"])
    drum_stick_right = plant.GetModelInstanceByName("drum_stick_right")
    drum_stick_left = plant.GetModelInstanceByName("drum_stick_left")
    right_frame = plant.GetFrameByName("stick",drum_stick_right )
    left_frame = plant.GetFrameByName("stick", drum_stick_left)


    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), 10)
    prog = trajopt.get_mutable_prog()
    trajopt.AddDurationCost(1.0)
    trajopt.AddPathLengthCost(1.0)
    trajopt.AddPositionBounds(
        plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()
    )
    trajopt.AddVelocityBounds(
        plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits()
    )
    plant_context = plant.GetMyMutableContextFromRoot(context)
    rightArm = False
    leftArm = False
    #get the ending posiiton of the IK solution from the initial position a couple cm above the drumkit target
    if hit["hit"] in right:
        rightArm = True
    else:
        leftArm = True

    if rightArm:
        print('Try to use right arm')
        end_effector_pose = plant.CalcRelativeTransform(
        plant_context,
        frame_A=plant.world_frame(),
        frame_B=right_frame
        )
        print('End effector pose is ', end_effector_pose)
        translation = np.array(centers[hit["hit"]])[:3]
        rotation = RotationMatrix(RollPitchYaw(np.array(centers[hit["hit"]])[3:]))
        X_WD = RigidTransform(rotation, translation)
        visualize_frame('object',X_WD )
        num_q = plant.num_positions()
        q0 = plant.GetPositions(plant_context)
        print('q0 is', q0)

        # start constraint
        start_constraint = PositionConstraint(
            plant,
            plant.world_frame(),
            end_effector_pose.translation(),
            end_effector_pose.translation(),
            right_frame,
            [0,0,0],
            plant_context,
        )
        goal_constraint = PositionConstraint(
            plant,
            plant.world_frame(),
            X_WD.translation() - np.array([0.02,0.02,0]),
            X_WD.translation() + np.array([0.02,0.02,0]),
            right_frame,
            [0,0,0],
            plant_context,
        )
    else:
        print('Try to use left arm')
        end_effector_pose = plant.CalcRelativeTransform(
        plant_context,
        frame_A=plant.world_frame(),
        frame_B=left_frame
        )
        translation = np.array(centers[hit["hit"]])[:3]
        rotation = RotationMatrix(RollPitchYaw(np.array(centers[hit["hit"]])[3:]))
        X_WD = RigidTransform(rotation, translation)
        visualize_frame('object',X_WD )
        num_q = plant.num_positions()
        q0 = plant.GetPositions(plant_context)

        # start constraint
        start_constraint = PositionConstraint(
            plant,
            plant.world_frame(),
            end_effector_pose.translation(),
            end_effector_pose.translation(),
            left_frame,
            [0,0,0.1],
            plant_context,
        )
        goal_constraint = PositionConstraint(
            plant,
            plant.world_frame(),
            X_WD.translation() - np.array([0.02,0.02,0]),
            X_WD.translation() + np.array([0.02,0.02,0]),
            left_frame,
            [0,0,0.1],
            plant_context,
        )

    print('Trying to set other constraints')
    trajopt.AddPathPositionConstraint(goal_constraint, 1)
    trajopt.AddPathPositionConstraint(start_constraint, 0)
    prog.AddQuadraticErrorCost(
        np.eye(num_q), q0, trajopt.control_points()[:, 0]
    )
    hit_velocity = 0.1*hit['strength']*np.array([1,1,1])

    plant_auto_diff = plant.ToAutoDiffXd() # make sure to call this after calling plant.finalize
    print('After diff call')
    final_vel_constraint = SpatialVelocityConstraint(
            plant_auto_diff,
            plant_auto_diff.world_frame(),
            hit_velocity,  # upper limit
            hit_velocity,  # lower limit
            plant_auto_diff.GetFrameByName("stick",drum_stick_right),
            np.array([0, 0, 0]).reshape(-1,1),
            plant_auto_diff.CreateDefaultContext(),)
    print('before result')
    result = Solve(prog)
    print('result is ', result)
    print(result.is_success())

    if not result.is_success():
        print("Trajectory optimization failed, even without collisions!")
        print(result.get_solver_id().name())
        continue
    trajopt.SetInitialGuess(trajopt.ReconstructTrajectory(result))
    if avoid_collisions:
        # collision constraints
        collision_constraint = MinimumDistanceLowerBoundConstraint(
            plant, 0.001, plant_context, None, 0.001
        )
        evaluate_at_s = np.linspace(0, 1, 25)
        for s in evaluate_at_s:
            trajopt.AddPathPositionConstraint(collision_constraint, s)

        def PlotPath(control_points):
            traj = BsplineTrajectory(
                trajopt.basis(), control_points.reshape((3, -1))
            )
            meshcat.SetLine(
                "positions_path", traj.vector_values(np.linspace(0, 1, 50))
            )

        result = Solve(prog)
        if not result.is_success():
            print("Trajectory optimization failed for collision free case")
            print(result.get_solver_id().name())
        PublishPositionTrajectory(
            trajopt.ReconstructTrajectory(result), context, plant, visualizer, hit["total_time"]
        )
        print('post publishPositionTrajectort')
        collision_visualizer.ForcedPublish(
            collision_visualizer.GetMyContextFromRoot(context)
        )
        print('post collisitonVisualizer')
    # simulator.AdvanceTo(hit["total_time"]) This line causes error 

meshcat.StopRecording() 
meshcat.PublishRecording()


hi_hat
Try to use left arm
Trying to set other constraints
After diff call
before result
result is  <pydrake.solvers.MathematicalProgramResult object at 0x7fc6781fc4b0>
True
post publishPositionTrajectort
post collisitonVisualizer
hi_hat
Try to use left arm
Trying to set other constraints
After diff call
before result
result is  <pydrake.solvers.MathematicalProgramResult object at 0x7fc650596570>
False
Trajectory optimization failed, even without collisions!
SNOPT
snare
Try to use left arm
Trying to set other constraints
After diff call
before result
result is  <pydrake.solvers.MathematicalProgramResult object at 0x7fc649c799f0>
False
Trajectory optimization failed, even without collisions!
SNOPT
hi_hat
Try to use left arm
Trying to set other constraints
After diff call
before result
result is  <pydrake.solvers.MathematicalProgramResult object at 0x7fc64c8a9170>
True
Trajectory optimization failed for collision free case
SNOPT
post publishPositionTrajectort
post collisitonVisualizer
hi

In [34]:
np.eye(np.shape(q0)[0])

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])